In [1]:
import pandas as pd    
import re
from name_prediction.code import bleu

### Testing data collection 

In [12]:
# testing Java code 
modelTest = "/Users/fjirigesi/Documents/Polyglot/name_prediction/dataset/java/test.jsonl"
modelTestDF = pd.read_json(path_or_buf=modelTest, lines=True)
# all tested code information 
allJavaTest = "/Users/fjirigesi/Documents/Polyglot/combine/test_java.jsonl"
allJavaTestDF = pd.read_json(path_or_buf=allJavaTest, lines=True)

# save original method name
allJavaTestDF['origin_method_name'] = modelTestDF['docstring_tokens']

In [3]:
allJavaTestDF.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition,origin_method_name
0,ReactiveX/RxJava,src/main/java/io/reactivex/internal/observers/...,QueueDrainObserver.fastPathOrderedEmit,protected final void fastPathOrderedEmit(U val...,java,protected final void fastPathOrderedEmit(U val...,"[protected, final, void, fastPathOrderedEmit, ...",Makes sure the fast-path emits in order.\n@par...,"[Makes, sure, the, fast, -, path, emits, in, o...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[fastPathOrderedEmit]
1,ReactiveX/RxJava,src/main/java/io/reactivex/Observable.java,Observable.wrap,@CheckReturnValue\n @SchedulerSupport(Sched...,java,@CheckReturnValue\n @SchedulerSupport(Sched...,"[@, CheckReturnValue, @, SchedulerSupport, (, ...",Wraps an ObservableSource into an Observable i...,"[Wraps, an, ObservableSource, into, an, Observ...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[wrap]
2,ReactiveX/RxJava,src/main/java/io/reactivex/Observable.java,Observable.sorted,@CheckReturnValue\n @SchedulerSupport(Sched...,java,@CheckReturnValue\n @SchedulerSupport(Sched...,"[@, CheckReturnValue, @, SchedulerSupport, (, ...",Returns an Observable that emits the events em...,"[Returns, an, Observable, that, emits, the, ev...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[sorted]
3,ReactiveX/RxJava,src/main/java/io/reactivex/internal/operators/...,ObservableReplay.observeOn,public static <T> ConnectableObservable<T> obs...,java,public static <T> ConnectableObservable<T> obs...,"[public, static, <, T, >, ConnectableObservabl...",Child Observers will observe the events of the...,"[Child, Observers, will, observe, the, events,...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[observeOn]
4,ReactiveX/RxJava,src/main/java/io/reactivex/processors/UnicastP...,UnicastProcessor.create,@CheckReturnValue\n @NonNull\n public st...,java,@CheckReturnValue\n @NonNull\n public st...,"[@, CheckReturnValue, @, NonNull, public, stat...",Creates an UnicastProcessor with the given int...,"[Creates, an, UnicastProcessor, with, the, giv...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[create]


### Extract predicted method names

In [4]:
def splitPuncts(line):
    return ' '.join(re.findall(r"[\w]+|[^\s\w]", line))

In [5]:
predict_file = "/Users/fjirigesi/Documents/Polyglot_result/output/test_1.output"

of = open(predict_file, 'r', encoding="utf-8")

predictMap = {}
idx = 0
predicted_method_names = []
for row in of:
    (rid, pred) = row.split('\t')
    predicted_method_names.append(pred.split('\n')[0])
    predictMap[rid] = []
    predictMap[rid].append(splitPuncts(pred.strip().lower()))

allJavaTestDF['predicted_method_names'] = predicted_method_names


### Calculate scores for prediected method names 

In [11]:
dev_gold_file = "/Users/fjirigesi/Documents/Polyglot_result/output/test_1.gold"

gf = open(dev_gold_file, 'r', encoding="utf-8")

goldMap = {}
idx = 0
original_method_names = []
for row in gf:
    (rid, ori) = row.split('\t') 
    original_method_names.append(ori.split('\n')[0])
    goldMap[rid] = []
    goldMap[rid].append(splitPuncts(ori.strip().lower()))

In [7]:
def calculate(original_name, predict_name):
    #split words
    x = original_name.split()
    x = list(original_name)
    y = predict_name.split()
    y = list(predict_name)
    #calculate common chars 
    common = []
    
    for ch in x:
        if ch in y:
            common.append(ch)
    # char in candidate, but not in predicted word 
    FN_list = []
    for w in x:
        if w not in common:
            FN_list.append(w)
    # char in predicted word, but not in candidate 
    FP_list=[]
    for w in y:
        if w not in common:
            FP_list.append(w)

    # calculate precision, recall and fscore
    TP=len(common)
    FP=len(FP_list)
    FN=len(FN_list)

    if TP==0:
        prec=0.0
        recall=0.0
        fscore=0.0
    else:
        prec=TP/(TP+FP)*1.0
        recall=TP/(TP+FN)*1.0
        fscore=2.0*prec*recall/(prec+recall)

    return (prec, recall, fscore)

In [8]:
precesions = []
recalls = []
fscores = []

number_samples = len(original_method_names)

for i in range(number_samples):
    prec, recall, fscore = calculate(original_method_names[i], predicted_method_names[i])
    precesions.append(prec)
    recalls.append(recall)
    fscores.append(fscore)
    
allJavaTestDF['precision'] = precesions
allJavaTestDF['recall'] = recalls
allJavaTestDF['fscore'] = fscores

In [9]:
allJavaTestDF

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition,origin_method_name,predicted_method_names,precision,recall,fscore
0,ReactiveX/RxJava,src/main/java/io/reactivex/internal/observers/...,QueueDrainObserver.fastPathOrderedEmit,protected final void fastPathOrderedEmit(U val...,java,protected final void fastPathOrderedEmit(U val...,"[protected, final, void, fastPathOrderedEmit, ...",Makes sure the fast-path emits in order.\n@par...,"[Makes, sure, the, fast, -, path, emits, in, o...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[fastPathOrderedEmit],add,1.000000,0.166667,0.285714
1,ReactiveX/RxJava,src/main/java/io/reactivex/Observable.java,Observable.wrap,@CheckReturnValue\n @SchedulerSupport(Sched...,java,@CheckReturnValue\n @SchedulerSupport(Sched...,"[@, CheckReturnValue, @, SchedulerSupport, (, ...",Wraps an ObservableSource into an Observable i...,"[Wraps, an, ObservableSource, into, an, Observ...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[wrap],of,0.000000,0.000000,0.000000
2,ReactiveX/RxJava,src/main/java/io/reactivex/Observable.java,Observable.sorted,@CheckReturnValue\n @SchedulerSupport(Sched...,java,@CheckReturnValue\n @SchedulerSupport(Sched...,"[@, CheckReturnValue, @, SchedulerSupport, (, ...",Returns an Observable that emits the events em...,"[Returns, an, Observable, that, emits, the, ev...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[sorted],sort,1.000000,0.571429,0.727273
3,ReactiveX/RxJava,src/main/java/io/reactivex/internal/operators/...,ObservableReplay.observeOn,public static <T> ConnectableObservable<T> obs...,java,public static <T> ConnectableObservable<T> obs...,"[public, static, <, T, >, ConnectableObservabl...",Child Observers will observe the events of the...,"[Child, Observers, will, observe, the, events,...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[observeOn],re play,0.600000,0.500000,0.545455
4,ReactiveX/RxJava,src/main/java/io/reactivex/processors/UnicastP...,UnicastProcessor.create,@CheckReturnValue\n @NonNull\n public st...,java,@CheckReturnValue\n @NonNull\n public st...,"[@, CheckReturnValue, @, NonNull, public, stat...",Creates an UnicastProcessor with the given int...,"[Creates, an, UnicastProcessor, with, the, giv...",ac84182aa2bd866b53e01c8e3fe99683b882c60e,https://github.com/ReactiveX/RxJava/blob/ac841...,test,[create],new Instance,0.416667,0.833333,0.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10950,ieb/sparsemapcontent,core/src/main/java/org/sakaiproject/nakamura/l...,Types.loadFromStream,"public static void loadFromStream(String key, ...",java,"public static void loadFromStream(String key, ...","[public, static, void, loadFromStream, (, Stri...",Load a Map from binary stream\n\n@param output...,"[Load, a, Map, from, binary, stream]",0570fdf868adbbf7734790cbc09fe66480e5f2dc,https://github.com/ieb/sparsemapcontent/blob/0...,test,[loadFromStream],read,1.000000,0.375000,0.545455
10951,intellimate/IzouSDK,src/main/java/org/intellimate/izou/sdk/addon/A...,AddOn.register,@Override\n public void register() {\n ...,java,@Override\n public void register() {\n ...,"[@, Override, public, void, register, (, ), {,...",This method is used to register the modules,"[This, method, is, used, to, register, the, mo...",bc8705ad48a6ca12a722f2b787be435949fa5d08,https://github.com/intellimate/IzouSDK/blob/bc...,test,[register],execute,0.500000,0.375000,0.428571
10952,intellimate/IzouSDK,src/main/java/org/intellimate/izou/sdk/addon/A...,AddOn.initAddOn,@Override\n public void initAddOn(org.intel...,java,@Override\n public void initAddOn(org.intel...,"[@, Override, public, void, initAddOn, (, org,...",Internal initiation of addOn - fake constructo...,"[Internal, initiation, of, addOn, -, fake, con...",bc8705ad48a

In [10]:
allJavaTestDF.to_csv('java_v1_result.csv')

# Parse Java Code

In [13]:
import javalang

In [41]:
code_sample = allJavaTestDF.iloc[0]['code']
print(code_sample)

protected final void fastPathOrderedEmit(U value, boolean delayError, Disposable disposable) {
        final Observer<? super V> observer = downstream;
        final SimplePlainQueue<U> q = queue;

        if (wip.get() == 0 && wip.compareAndSet(0, 1)) {
            if (q.isEmpty()) {
                accept(observer, value);
                if (leave(-1) == 0) {
                    return;
                }
            } else {
                q.offer(value);
            }
        } else {
            q.offer(value);
            if (!enter()) {
                return;
            }
        }
        QueueDrainHelper.drainLoop(q, observer, delayError, disposable, this);
    }


In [71]:
tokens = javalang.tokenizer.tokenize(code_sample)
parser = javalang.parser.Parser(tokens)
print(parser)
type(parser)

javalang.parser.Parser

In [80]:
methods = dir(parser)

In [81]:
methods = methods[26:]

In [82]:
methods

['accept',
 'build_binary_operation',
 'debug',
 'illegal',
 'is_annotation',
 'is_annotation_declaration',
 'operator_precedence',
 'parse',
 'parse_annotation',
 'parse_annotation_element',
 'parse_annotation_method_or_constant_rest',
 'parse_annotation_type_body',
 'parse_annotation_type_declaration',
 'parse_annotation_type_element_declaration',
 'parse_annotation_type_element_declarations',
 'parse_annotations',
 'parse_arguments',
 'parse_array_creator_rest',
 'parse_array_dimension',
 'parse_array_initializer',
 'parse_basic_type',
 'parse_block',
 'parse_block_statement',
 'parse_catch_clause',
 'parse_catches',
 'parse_class_body',
 'parse_class_body_declaration',
 'parse_class_creator_rest',
 'parse_class_or_interface_declaration',
 'parse_compilation_unit',
 'parse_constant_declarator',
 'parse_constant_declarator_rest',
 'parse_constant_declarators_rest',
 'parse_constructor_declarator_rest',
 'parse_created_name',
 'parse_creator',
 'parse_element_value',
 'parse_element_v

In [83]:
parser.parse_annotations()

JavaSyntaxError: 

In [58]:
tree.is_annotation()

False

In [19]:
tokens = javalang.tokenizer.tokenize('System.out.println("Hello " + "world");')

In [23]:
parser = javalang.parser.Parser(sample)

In [24]:
parser

In [26]:
parser

In [27]:
parser.parse_type_declaration()

AttributeError: 'str' object has no attribute 'value'